In [4]:
import pandas as pd 
import seaborn as sns
from scipy import stats 
import seaborn as sns 
import matplotlib.pyplot as plt
from feature_engine.selection import DropCorrelatedFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np 
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV , KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor 
import plotly.graph_objects as go
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import BayesianRidge
from numpy import arange
from xgboost.sklearn import XGBRegressor
import lightgbm as lgb
import statsmodels.api as sm 

###### IMPORT DATA 

In [5]:
df = pd.read_excel('trainDataset.xls')
df = df.drop('ID', axis=1)
df = df.drop('pCR (outcome)', axis = 1)

## DATA PREPROCESSING

###### imputing missing values with median 

In [6]:
imp = SimpleImputer(missing_values=999, strategy='median')
# Fitting the data to the imputer object
imputer = imp.fit(df)
imputed_df = imputer.transform(df)
imputed_df = pd.DataFrame(imputed_df, columns = df.columns)



###### Train Test Split

In [7]:
rfs = imputed_df['RelapseFreeSurvival (outcome)']
features = imputed_df.drop('RelapseFreeSurvival (outcome)', axis =1 )
x_train, x_test, y_train, y_test = train_test_split(features,rfs, test_size = 0.1, random_state = 42)


###### read test set data

In [8]:
test = pd.read_excel('testDatasetExample.xls')
patient_ID  = test['ID']
test_set = test.drop('ID', axis=1)

###### feature normalisation

In [9]:
scaler = MinMaxScaler()
scaled_x = scaler.fit_transform(x_train)
scaled_train_x = pd.DataFrame(scaled_x)
x_test = scaler.fit_transform(test_set)

###### PCA


In [10]:
pca = PCA(n_components=11)
PCA_fit = pca.fit(scaled_train_x)
x_PCA = PCA_fit.transform(scaled_train_x)
x_test =PCA_fit.transform(x_test)

### Fitting the model with best parameters

###### Bayesian Ridge Regression


In [11]:
model = BayesianRidge(alpha_init = 0.3 , lambda_init = 1e-09)
model.fit(x_PCA, y_train)
pred = model.predict(x_test)

BayesianRidge(alpha_init=0.3, lambda_init=1e-09)

###### Put results in data frame 

In [13]:
pred = pd.DataFrame(pred)
ID = pd.DataFrame(patient_ID)
pred_with_ID = pd.concat([ID, pred], axis = 1)
# rename outcome column
pred_with_ID.rename( columns = { 0:'RelapseFreeSurvival (outcome)'}, inplace=True )

###### output to csv file

In [14]:
pred_with_ID.to_csv('RFSPrediction.xls')